In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import math
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Found and verified text8.zip
Data size 100000000
99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl
Unexpected character: ï
1 26 0 0
a z  


In [30]:
batch_size=64
num_unrollings=10
embedding_size=27

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()

  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size,2), dtype=np.int32)
    for b in range(self._batch_size):
      batch[b,0] = char2id(self._text[self._cursor[b]])
      batch[b,1] = char2id(self._text[self._cursor[b]+1])
      self._cursor[b] = (self._cursor[b] +2) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(batches):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  batches=list(map(list, zip(*batches)))
  s=list("")
  for b in batches:
        ss=""
        for c in b:
            ss+=id2char(int(c[0]))
            ss+=id2char(int(c[1]))
        s.append(ss)
  return s

def characters2(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  s=[id2char(np.floor_divide(c,27)) for c in np.argmax(probabilities, 1)]
  s+=[id2char(c-27*np.floor_divide(c,27)) for c in np.argmax(probabilities, 1)]
  return s

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print((np.array(train_batches.next())).shape)
print(characters(train_batches.next()))
print(characters(train_batches.next()))
print(characters(valid_batches.next()))
print(characters(valid_batches.next()))

# ==========================
# OTHER EVALUATION FUNCTIONS
# ==========================

def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 26.0, size=[1, 1])
  return b[:,None]

(11, 64, 2)
['ate social relations b', 'ments failed to revive', 'al park photographic v', 'ies index sacred desti', 'ess of castile daughte', ' h provided a detailed', 'guage among jews manda', 'gers in december one n', 'al media and from pres', ' during the one nine e', 'known manufacturers of', 'seven a widebody jet w', 's covering some of the', 'en one of the most inf', 'ze single acts of meri', ' first card from the d', ' in jersey and guernse', 'he poverty and social ', 'gns of humanity vol th', ' cause so aquinas come', 'n denaturalization and', 'ce formation solution ', 'the input usually meas', 'ck to pull him out but', 'usion inability to ori', 'omplete an operation c', 't of the mistakes of a', ' it fort des moines th', 'ttempts by his opponen', 'ormats for mailboxes i', 'soteric christianity a', 'growing popularity of ', 'riginal document fax m', 'e nine eight zero one ', 'rch eight listing of a', 'haracter lieutenant sh', 'al mechanics and speci', ' gm comparison maize c',

In [24]:
num_nodes = 64
vocabulary_size=27*27

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # first parameter of each gate in 1 matrix:
  i_all = tf.Variable(tf.truncated_normal([embedding_size, 4*num_nodes], -0.1, 0.1))
  # second parameter of each gate in 1 matrix
  o_all = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    
  # Input gate: input, previous output, and bias.
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases. 
 
 
  #embeddings
  embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size],-0.1,0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_mat=tf.matmul(i,i_all)
    output_mat=tf.matmul(o,o_all)
    
    input_gate = tf.sigmoid(input_mat[:,0:num_nodes] + output_mat[:,0:num_nodes] + ib)
    forget_gate = tf.sigmoid(input_mat[:,num_nodes:2*num_nodes] + output_mat[:,num_nodes:2*num_nodes] + fb)
    update = input_mat[:,2*num_nodes:3*num_nodes] + output_mat[:,2*num_nodes:3*num_nodes] + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(input_mat[:,3*num_nodes:] + output_mat[:,3*num_nodes:] + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings+1):
    train_data.append(tf.placeholder(tf.int32, shape=[batch_size,2]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    i_concat=27*i[:,0]+i[:,1]
    embedded_i=tf.nn.embedding_lookup(embeddings,i_concat)
    output, state = lstm_cell(embedded_i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    outputs_concat=tf.concat(outputs,0)
    #try to compute similarity here as well
    logits=tf.nn.xw_plus_b(outputs_concat,w,b)
    print((logits).shape)
    
    #Compute one hot encodings
    label_batch=tf.concat(train_labels,0)    
    label_batch=27*label_batch[:,0]+label_batch[:,1]
    print(label_batch.shape)
    sparse_labels = tf.reshape(label_batch, [-1, 1])
    derived_size = tf.shape(label_batch)[0]
    indices = tf.reshape(tf.range(0, derived_size, 1), [-1, 1])
    print(indices.shape,'indices.shape')
    concated = tf.concat([indices, sparse_labels],1)
    outshape = tf.stack([derived_size, vocabulary_size])
    labels = tf.sparse_to_dense(concated, outshape, 1.0, 0.0)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 10000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  print(train_prediction.shape)
  
  # Sampling and validation eval: batch 1, no unrolling.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  #valid_dataset=np.array([i for i in range(27)])
  #valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
 
  sample_input = tf.placeholder(tf.int32, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  embedded_sample=tf.nn.embedding_lookup(embeddings,sample_input)
  sample_output, sample_state = lstm_cell(embedded_sample, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    #similarity = tf.matmul(sample_prediction, tf.transpose(normalized_embeddings))
    print(sample_prediction.shape)

(640, 729)
(640,)
(?, 1) indices.shape
(640, 729)
(1, 729)


In [37]:
num_steps = 50001
summary_frequency = 500

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
      #print((feed_dict[train_data[i]]).shape)
    
    _, l, predictions, lr,train_lab = session.run([optimizer, loss, train_prediction, learning_rate,train_labels], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      
      #print(train_lab)
      #print(labels.shape[0])
      #print('Minibatch perplexity: ',np.exp(logprob(predictions, labels)))
      
      if step % (summary_frequency * 2) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = np.zeros(shape=(1,), dtype=np.int32)
          feed[0,] =np.random.randint(0,729)
          sentence=id2char(np.floor_divide(feed[0],27))
          sentence+=id2char(feed[0]-27*np.floor_divide(feed[0],27)) 
          reset_sample_state.run()
          for _ in range(50):
            prediction=sample_prediction.eval({sample_input:feed})
            k = sample(prediction)
            k=characters2(k)
            #print(k)
            feed = np.zeros(shape=(1,), dtype=np.int32)
            feed[0,] = 27*char2id(k[0])+char2id(k[1])
            sentence += k[0]
            sentence+=k[1]
            #feed = np.zeros(shape=(1,), dtype=np.int32)
            #feed[0,] = np.argmax(prediction)
            #print(feed.shape)
            #sentence += id2char(feed[0,])
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      #for _ in range(valid_size):
       # b = valid_batches.next()
        #predictions = sample_prediction.eval({sample_input: b[0]})
        #valid_logprob = valid_logprob + logprob(predictions, b[1])
      #print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.606374 learning rate: 10.000000
ljzlsvljsvgzmgkvftykpepqtjlxuksofmsexiuiwcbdo ichhoxubhybsbxkxchwfnmjkbppywevjds qrulzpekgsv wqtbadfzy
xphgihjqkmmuoreeavftwujicfsaertdfdejuxkyfsxawxpebvvmteojafgkhpoqjgsaripqdomtzgkxyaxsnzitbxhnnvdji vylf
kdqhtuuyldbgmgwkvkervbrsoyqorpxyleujt boh stnvgogxddimnbniiq ccghyjcrrktuitgrscwtpdiyiebqtmcpoykcayqbs
ocapvbvwoymhigdxsxbcpvdtnzyp tdhvjcbckdyut npztahdchzckezcnxhxffmpaleliizosyaabtsfqulndjmazpekllkvctcp
fatxjmghsejawkrqxnvievjactjdjbtbxniv idfammcwcnhfphhqaziuajlkxt wksu  vgwwfqrvzbfnjmqayclicposawoxth u
Average loss at step 500: 4.155773 learning rate: 10.000000
Average loss at step 1000: 3.490247 learning rate: 10.000000
uvadnownise is the great from as a mytholotert bovemen the world yold by cresentemred invotitud and se
zh se in the element of is beiniture evential its the easq magainalony os on lifeltitu unid tezuher my
rlzwll donists one offolading apange anlivora war the vvnyte sfarpresulimane of civil ph